## Exercice 2  : _Images bitmap au format PBM_

###  Une première solution

In [25]:
L, H = 500, 500
f = open('bandes.pbm', 'w')
f.write('P1\n')
f.write(str(L) + ' ' + str(H) + '\n')
for lig in range(H):
    for col in range(L):
        if (lig // 50 + col // 50)%2 == 0 and lig % 50 + col % 50 <= 49 \
        or (lig // 50 + col // 50)%2 == 1 and lig % 50 + col % 50 > 49 :
            f.write('0') #blanc
        else:
            f.write('1') #noir           
    f.write('\n')
f.close()

### Une solution plus compacte

In [26]:
L, H = 500, 500
f = open('bandes.pbm', 'w')
f.write('P1\n')
f.write(str(L) + ' ' + str(H) + '\n')
for lig in range(H):
    for col in range(L):
        if ((lig + col) // 50)%2 == 1:
            f.write('1') #noir
        else:
            f.write('0') #blanc          
    f.write('\n')
f.close()

In [36]:
#affichage avec wand module de Python de binding pour ImageMagick
def conversion(imagesource,imagebut, formatbut):
    import wand
    from wand.image import Image
    from wand.display import display
    #on convertit l'image inversée en PNG avec wand
    impbm = wand.image.Image(filename = imagesource)
    impng = impbm.convert(formatbut)
    impng.save(filename = imagebut)
    
conversion('bandes.pbm', 'bandes-pbm.png','png')

![](bandes-pbm.png)

## Exercice 3  : _Images bitmap au format PGM_

In [28]:
def inverser_couleurs_pbm(source, but):
    """Lit le fichier pbm source et le recopie dans le fichier pgm
    but en inversant les couleurs"""
    #ouverture de fichiers en lecture pour source et écriture pour but
    f = open(source, 'r')
    g = open(but, 'w')
    #on recopie l'en-tête (les deux premières lignes)
    for k in range(2):
        lig = f.readline()
        g.write(lig)
    #pour les lignes codant les pixels on inverse chaque valeur de pixel
    for lig in f:
        for caractere in lig:
            if caractere == '0':
                g.write('1')
            elif caractere == '1':
                g.write('0')
            else:
                g.write(caractere)
    g.close()
    f.close()

In [29]:
inverser_couleurs_pbm('damier.pbm', 'damier-inverse.pbm')

In [30]:
conversion('damier-inverse.pbm',  'damier-inverse.png', 'png')

![](damier-inverse.png)

## Exercice 3 _Images en niveaux de gris au format PGM_

In [31]:
def rechercher_pixel_ppm(source, lig, col):
    """Retourne la valeur du pixel en ligne lig et colonne col 
    dans le fichier image source au format PGM"""
    f = open(source, 'r')
    for k in range(3 + lig + 1):
        f.readline()
    liste_pixels = f.readline().split()
    f.close()
    return int(liste_pixels[col])
    
def flip_ppm(source, but):
    """Retourne un fichier but au format PGM obtenu en appliquant une symétrie 
    d'axe vertical (flip) à l'image du fichier source au format PGM"""
    #ouverture de fichiers en lecture pour source et écriture pour but
    f = open(source, 'r')
    g = open(but, 'w')
    #on recopie l'en-tête (les trois premières lignes)
    for k in range(3):
        lig = f.readline()
        g.write(lig)
    for lig in f:
        liste_pixels = lig.split()
        g.write(' '.join(liste_pixels[::-1]) + '\n')
    g.close()
    f.close()
    
def flop_ppm(source, but):
    """Retourne un fichier but au format PGM obtenu en appliquant une symétrie 
    d'axe horizontal (flop) à l'image du fichier source au format PGM"""
    #ouverture de fichiers en lecture pour source et écriture pour but
    f = open(source, 'r')
    g = open(but, 'w')
    #on recopie l'en-tête (les trois premières lignes)
    for k in range(3):
        lig = f.readline()
        g.write(lig)
    liste_lignes = f.readlines()
    for lig in reversed(liste_lignes):
        g.write(lig)
    g.close()
    f.close()

In [32]:
flip_ppm('lena.ppm', 'lena-flip.ppm')
flop_ppm('lena.ppm', 'lena-flop.ppm')

In [33]:
conversion('lena.ppm', 'lena.png', 'png')
conversion('lena-flip.ppm', 'lena-flip.png', 'png')
conversion('lena-flop.ppm', 'lena-flop.png', 'png')

### lena.png

![](lena.png)

### lena-flip.png

![](lena-flip.png)

### lena-flop.png

![](lena-flop.png)

## Exercice 3 Fichiers binaires et images bitmap

Un premier exemple d'introduction

In [1]:
from PIL import Image
import numpy


#on ouvre l'image
im = Image.open('lenagray.png')

#on affiche les dimensions, le format de fichier et le mode de l'image (binaire, niveaux de gris (L), RGB)
print(im.size,im.format, im.mode)

#on transforme l'image en tableau numpy à 2 dimensions  de pixels    
tab = numpy.asarray(im)

#on affiche le pixel en première ligne et dernière colonne
print(tab[0][511])

#on affiche les vingt premiers pixels de la première ligne
print(tab[0][:20])

#on affiche le pixel en dernière ligne et première colonne
print(tab[511][0])

#un flop

#on crée un tableau 2 de mêmes dimensions que tab et rempli de 0
#Attention à bien créer un tableau numpy et à préciser le type 'uint8' qui correspond à un octer par pixel
nblig, nbcol = len(tab), len(tab[0])
tab2 = numpy.array([ [0] * nbcol for _ in range(nblig) ], dtype ='uint8')

#on recopie tab dans tab 2 mais en inversant l'ordre des lignes  (flop)
for lig in range(0, nblig):
    for col in range(0, nbcol):
        tab2[lig][col] = tab[nblig - 1 - lig][col]
        
#on transforme tab2 en image
im2 = Image.fromarray(tab2)
#on enregistre cette image sur le disque
im2.save("lenagray-flop.png")

#Un flip 

#on crée un tableau 2 de mêmes dimensions que tab et rempli de 0
#Attention à bien créer un tableau numpy et à préciser le type 'uint8' qui correspond à un octer par pixel
nblig, nbcol = len(tab), len(tab[0])
tab3 = numpy.array([ [0] * nbcol for _ in range(nblig) ], dtype ='uint8')

#on recopie tab dans tab 2 mais en inversant l'ordre des lignes  (flop)
for lig in range(0, nblig):
    for col in range(0, nbcol):
        tab3[lig][col] = tab[lig][nbcol - 1 - col]
        
#on transforme tab2 en image
im3 = Image.fromarray(tab3)
#on enregistre cette image sur le disque
im3.save("lenagray-flip.png")

#passons au négatif
def negatif_gris(source, but):
    """Enregistre sur le disque un négatif de l'image source en niveaux de gris"""
    imsource= Image.open(source)
    tab = numpy.asarray(imsource)
    nbcol, nblig = len(tab[0]), len(tab)
    tab2 = numpy.array([ [0] * nbcol for _ in range(nblig) ], dtype ='uint8')    
    for lig in range(nblig):
        for col in range(nbcol):
            tab2[lig][col] = 255 - tab[lig][col]
    imbut = Image.fromarray(tab2)
    imbut.save(but)

negatif_gris('lenagray.png', 'lenagray-negatif.png')

(512, 512) PNG L
129
[162 162 164 162 161 157 163 161 164 160 163 160 153 161 159 154 154 155
 160 158]
53


Transformation du photomaton

In [34]:
## Imports des modules

from PIL import Image
import numpy


##Exercice 3 Question 1

def transformation(i, j, n):
    """Retourne les coordonnées (colonne, ligne) de l'image du pixel de 
    coordonnées (i, j) par la transformation du photomaton"""
    if i % 2 == 0:
        if j % 2 == 0:
            return (i // 2, j // 2)
        else:
            return (i // 2, (j + n) // 2)
    else:
        if j % 2 == 0:
            return ((i + n) // 2, j // 2)
        else:
            return ((i + n) // 2, (j + n) // 2)
            
##Exercice 3 Question 2


def copie(tableau):
    """Retourne la copie profonde d'un tableau à deux dimensions"""
    return numpy.array([[tableau[j][i] for i in range(len(tableau[j]))] 
    for j in range(len(tableau))], dtype = 'uint8')
            

def photomaton(tableau):
    """Retourne le tableau calculé obtenu après application de la transformation
    du photomaton au tableau passé en paramètre"""
    tableau2 = copie(tableau)
    n = len(tableau)
    for i in range(n):
        for j in range(n):
            (i2, j2) = transformation(i, j, n)
            tableau2[i2][j2] = tableau[i][j]
    return tableau2
    
##Exercice 3 Question 3

def photomaton_iterer(source, k):
    im = Image.open(source)
    nom, extension = source.split('.')
    tab = numpy.asarray(im)
    for iteration in range(1, k + 1):
        tab = photomaton(tab)
        im = Image.fromarray(tab)
        im.save(nom + '-photomaton-iteration-' + str(iteration) + '.' + extension)

In [35]:
photomaton_iterer('lenagray-256.png', 8)

### Itération 1

![](lenagray-256-photomaton-iteration-1.png)

### Itération 2

![](lenagray-256-photomaton-iteration-2.png)

### Itération 3

![](lenagray-256-photomaton-iteration-3.png)

### Itération 4

![](lenagray-256-photomaton-iteration-4.png)

### Itération 5

![](lenagray-256-photomaton-iteration-5.png)

### Itération 6

![](lenagray-256-photomaton-iteration-6.png)

### Itération 7

![](lenagray-256-photomaton-iteration-7.png)

### Itération 8

![](lenagray-256-photomaton-iteration-8.png)
